In [1]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters docx2txt langchain-chroma

Note: you may need to restart the kernel to use updated packages.


## 2. Knowledge Base 구성을 위한 데이터 생성 
- RecursiveCharacterTextSplitter를 활용해서 데이터 chunking 
	- Recursively의 경우 list type
- chunk_size: split된 chunk's max size
- chunk_overlap: 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [2]:
# from langchain_community.document_loaders import Docx2txtLoader

# loader = Docx2txtLoader('./medical.docx')
# document = loader.load()

In [3]:
# document

In [4]:
# len(document) # 통으로 불러옴

## 3. 문서 쪼개기 (RecursiveCharacterTextSplitter)

단순히 로컬 파일('./medical.docx')을 읽어 텍스트 데이터를 처리하기 때문에 환경변수가 필요하지 않음

In [5]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
)

loader = Docx2txtLoader('./medical.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [6]:
len(document_list)

69

UpstageEmbeddings 객체를 초기화하는 과정에서 API 키나 인증 정보를 필요로 하기 때문에 환경변수 로드가 필요

In [7]:
from dotenv import load_dotenv # 환경변수 로드 
from langchain_upstage import UpstageEmbeddings

load_dotenv()

# Solar Pro will stay free until charging begins on Jan 6, 2025.
embedding = UpstageEmbeddings(model="embedding-query") 

- collection_name
	- 서로 다른 유형의 데이터를 독립적으로 관리하기 위해 사용
	- 특정 그룹의 데이터만 검색할 수 있어 검색 속도와 효율성 향상
	- 데이터를 추가하거나 삭제할 때도 사용 (특정 컬렉션만 삭제 혹은 업데이트 가능) 
	- 따라서, 데이터 관리 유용

In [8]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 (persis_directory: 날아가지 않도록)
database = Chroma.from_documents(documents=document_list,
                                 embedding=embedding,
                                 collection_name='medical-law',
                                 persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때 
# database = Chroma(collection_name='medical-law', persist_directory="./chroma", embedding_function=embedding)

In [9]:
query = '환자가 진료 기록 열람을 요청할 수 있는 절차는 무엇인가요?'

# 'k'값을 조정해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [10]:
retrieved_docs 

[Document(metadata={'source': './medical.docx'}, page_content='② 제58조제2항에 따라 의료기관 인증에 관한 업무에 종사하는 자 또는 종사하였던 자는 그 업무를 하면서 알게 된 정보를 다른 사람에게 누설하거나 부당한 목적으로 사용하여서는 아니 된다.<신설 2016. 5. 29.>\n\n[제목개정 2016. 5. 29.]\n\n\n\n제20조(태아 성 감별 행위 등 금지) ①의료인은 태아 성 감별을 목적으로 임부를 진찰하거나 검사하여서는 아니 되며, 같은 목적을 위한 다른 사람의 행위를 도와서도 아니 된다.\n\n②의료인은 임신 32주 이전에 태아나 임부를 진찰하거나 검사하면서 알게 된 태아의 성(性)을 임부, 임부의 가족, 그 밖의 다른 사람이 알게 하여서는 아니 된다.<개정 2009. 12. 31.>\n\n[2009. 12. 31. 법률 제9906호에 의하여 2008. 7. 31. 헌법재판소에서 헌법불합치 결정된 이 조 제2항을 개정함.]\n\n[단순위헌, 2022헌마356, 2024.2.28, 의료법(2009. 12. 31. 법률 제9906호로 개정된 것) 제20조 제2항은 헌법에 위반된다.]\n\n\n\n제21조(기록 열람 등) ① 환자는 의료인, 의료기관의 장 및 의료기관 종사자에게 본인에 관한 기록(추가기재ㆍ수정된 경우 추가기재ㆍ수정된 기록 및 추가기재ㆍ수정 전의 원본을 모두 포함한다. 이하 같다)의 전부 또는 일부에 대하여 열람 또는 그 사본의 발급 등 내용의 확인을 요청할 수 있다. 이 경우 의료인, 의료기관의 장 및 의료기관 종사자는 정당한 사유가 없으면 이를 거부하여서는 아니 된다. <신설 2016. 12. 20., 2018. 3. 27.>\n\n②의료인, 의료기관의 장 및 의료기관 종사자는 환자가 아닌 다른 사람에게 환자에 관한 기록을 열람하게 하거나 그 사본을 내주는 등 내용을 확인할 수 있게 하여서는 아니 된다.<개정 2009. 1. 30., 2016. 12. 20.>\n\n③ 제2항에도 

## 4. Augmentation을 위한 Prompt 활용
- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-promt") 사용

In [11]:
# RLM: Retrieval Language Model (검색 + 언어 모델의 결합)
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model="solar-pro")

In [12]:
from langchain import hub

# 미리 정의된 프롬프트 템플릿을 가져오는 역할
prompt = hub.pull("rlm/rag-prompt")

c:\Users\carly\anaconda3\envs\medical\lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## 5. 답변 생성
- RetrievalQA를 통해 LLM에 전달

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever = database.as_retriever(),
    chain_type_kwargs = {"prompt": prompt}
)

### as_retriever(): pipecope 등 다양한 곳에서 사용가능 
- LangChain에서 데이터베이스나 벡터 스토어를 검색기(retriever)로 변환하는 메소드

In [14]:
ai_message = qa_chain.invoke({"query": query})

In [15]:
ai_message

{'query': '환자가 진료 기록 열람을 요청할 수 있는 절차는 무엇인가요?',
 'result': '환자는 의료인, 의료기관의 장 및 의료기관 종사자에게 본인에 관한 기록(추가기재ㆍ수정된 경우 추가기재ㆍ수정된 기록 및 추가기재ㆍ수정 전의 원본을 모두 포함한다. 이하 같다)의 전부 또는 일부에 대하여 열람 또는 그 사본의 발급 등 내용의 확인을 요청할 수 있습니다. 의료인, 의료기관의 장 및 의료기관 종사자는 환자의 요청이 있을 경우 정당한 사유가 없는 한 진료 기록 열람을 허용해야 합니다.'}